In [1]:
import pandas as pd
import torch
import random
import numpy as np
from torch import nn

fix_seed = 42
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

pd.set_option('display.max_columns', None)
raw_data = pd.read_csv('./dataset/my/trimmed_LX3_ss0_se100_cr05_C_V_T_vs_CE.csv')

In [2]:
raw_data#.loc[(data['Cycle_Index']==5) & (data['battery_id']==1)]

,battery_id,Cycle_Index,cycle_test_time,leixin_LFP,leixin_NCA,leixin_NMC,Current (A),Voltage (V),Environment_Temperature (C),Cell_Temperature (C),Charge_Energy (mWh)
0,0,5.0,0.000,1.0,0.0,0.0,0.549,2.986,15.117,17.935,51.0
1,0,5.0,120.019,1.0,0.0,0.0,0.549,3.123,15.255,17.537,107.0
2,0,5.0,240.037,1.0,0.0,0.0,0.550,3.205,15.255,17.154,165.0
3,0,5.0,360.052,1.0,0.0,0.0,0.549,3.262,15.179,16.847,225.0
4,0,5.0,480.062,1.0,0.0,0.0,0.549,3.288,15.179,16.541,285.0
...,...,...,...,...,...,...,...,...,...,...,...
5448847,85,768.0,6259.386,0.0,0.0,1.0,0.378,4.200,34.888,35.213,8981.0
5448848,85,768.0,6379.388,0.0,0.0,1.0,0.331,4.200,34.867,35.101,9030.0
5448849,85,768.0,6379.391,0.0,0.0,1.0,0.331,4.200,34.827,35.101,9030.0
5448850,85,768.0,6979.266,0.0,0.0,1.0,0.000,4.147,34.787,34.924,9030.0


In [171]:
from sklearn.preprocessing import StandardScaler

seq_len = 18    # 训练长度
label_len = 9  # 重叠部分,用于预测序列回看知识
pred_len = 30   # 预测
set_type = 0
scale = True
drop_bid = False    # TODO:battery_id可能需要独热编码,考虑drop或者不进行归一化



# 按照电池+轮次打乱
indexed_df = raw_data.set_index(['battery_id', 'Cycle_Index'])
indexed_df['luncishu'] = raw_data.groupby(['battery_id', 'Cycle_Index']).count()['cycle_test_time']
indexed_df = indexed_df[indexed_df['luncishu'] > 48]
indexed_df = indexed_df.drop('luncishu', axis=1)
shuffled_index = indexed_df.index.unique().values
random.shuffle(shuffled_index)
shuffled_index = pd.Index(shuffled_index)

# 找到对应数据集的位置
border1s = [0, 0.6, 0.7]
border2s = [0.6, 0.7, 1]
def get_data(df, shuffled_index, a, b):
    border1 = int(a*shuffled_index.shape[0])
    border2 = int(b*shuffled_index.shape[0])
    flag = shuffled_index[border1:border2]
    df = df.loc[flag]
    return df

# 获取对应模式的数据
df = get_data(indexed_df, shuffled_index, border1s[set_type], border2s[set_type])

# 记录恢复标识
recover_index = df.index
recover_index_list = []
recover_name_list = []
a = pd.DataFrame(range(recover_index.shape[0]), index=recover_index)
groups = a.groupby(level=[0,1], sort=False)
for name, group in groups:
    recover_name_list.append(name)
    recover_index_list.append(group[0].to_numpy())
recover_index_list = np.array(recover_index_list, dtype=object)
recover_name_list = np.array(recover_name_list, dtype=object)

# unstack index
df = df.reset_index()   

# 删除bid
if drop_bid:
    df = df.drop('battery_id', axis=1)

# 归一化
scaler = StandardScaler()
if scale:
    train_data = get_data(indexed_df, shuffled_index, border1s[0], border2s[0])
    train_data = train_data.reset_index()
    if drop_bid:
        train_data = train_data.drop('battery_id', axis=1)
    scaler.fit(train_data.values)
    data = scaler.transform(df.values)  # 序列的特征值
else:
    data = df.values

data = np.array([data[index] for index in recover_index_list], dtype=object)


In [5]:
a = indexed_df.groupby(['battery_id', 'Cycle_Index']).count()['cycle_test_time']

a.value_counts()

cycle_test_time
58    13503
59    12499
57    11363
56     8995
61     8283
60     6905
55     6335
62     3780
53     3771
54     3634
52     2561
51     2066
63     1936
50     1680
49     1435
48     1217
64     1159
65      640
66      431
67      304
68      174
69       79
70       26
71        5
79        1
72        1
85        1
Name: count, dtype: int64

In [348]:
l = 200
ans = 0
for i, index in enumerate(recover_index_list):
    if index.shape[0] <= l:
        l = index.shape[0]
        ans = i

In [174]:
data.shape

(54940,)

In [178]:
for i in range(54940):
    seed = i   # 55858
    seq_len = 18    # 训练长度
    label_len = 9  # 重叠部分,用于预测序列回看知识
    pred_len = 30   # 预测


    insample = np.zeros((seq_len, 11))
    insample_mask = np.zeros((seq_len, 11))
    outsample = np.zeros((label_len + pred_len, 1))
    outsample_mask = np.zeros((label_len + pred_len, 1))


    seq = data[seed]
    # print(seq.shape)

    low = seq_len
    high = len(seq) - pred_len

    if low >= high:
        print(i)
    cut_point = np.random.randint(low=low,
                                high=high,
                                size=1)[0]    # 切分点,

    # print(cut_point)

    # 输入padding mask
    insample = seq[cut_point - seq_len: cut_point]

    # print(insample.shape)

    # 输出padding mask
    outsample = seq[cut_point-label_len: cut_point + pred_len]


    # print(max(1, cut_point-label_len), min(len(seq), cut_point + pred_len))
    # print(outsample.shape)

In [344]:
print(seq)

[[ 0.52245322 -0.47970737 -1.69700616 -1.71554209  2.90539484 -0.41627348
   2.14478976  2.29624503  0.18441992  0.4652107  -1.10469288]
 [ 0.52245322 -0.47970737 -1.63727047 -1.71554209  2.90539484 -0.41627348
  -0.82365808  2.2928561   0.19489179  0.45388876 -1.06716906]
 [ 0.52245322 -0.47970737 -1.57751388 -1.71554209  2.90539484 -0.41627348
  -1.12847499  2.28946716  0.18441992  0.38625507 -1.05067507]
 [ 0.52245322 -0.47970737 -1.51775728 -1.71554209  2.90539484 -0.41627348
  -1.257436    2.2928561   0.17808168  0.30014874 -1.03954163]
 [ 0.52245322 -0.47970737 -1.45800019 -1.71554209  2.90539484 -0.41627348
  -1.32777836  2.2928561   0.20343463  0.25962812 -1.03046994]
 [ 0.52245322 -0.47970737 -1.39824359 -1.71554209  2.90539484 -0.41627348
  -1.37467327  2.28946716  0.17601486  0.20763658 -1.02263529]
 [ 0.52245322 -0.47970737 -1.338487   -1.71554209  2.90539484 -0.41627348
  -1.4121892   2.2928561   0.20136782  0.2009328  -1.0160377 ]
 [ 0.52245322 -0.47970737 -1.2787304  -1.

In [7]:
raise ValueError(123124)

ValueError: 123124

In [4]:
from data_provider.data_factory import Dataset_Battery_mask
from torch.utils.data import DataLoader

mydataset = Dataset_Battery_mask(
    root_path='./dataset/my/',
    data_path='trimmed_LX3_ss0_se100_cr05_C_V_T_vs_CE.csv',

)
data_loader = DataLoader(
    mydataset,
    batch_size=64,
    shuffle=True,
    num_workers=1,
    drop_last=True)



for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(data_loader):
    print(1)
    break


AttributeError: 'Dataset_Battery_mask' object has no attribute 'seq_l'